# Using of Processor

In [1]:
from processor import Preprocessor
from FlagEmbedding import BGEM3FlagModel

In [2]:
preprocessor = Preprocessor(filepath="ai_doc.pdf")

In [3]:
text = preprocessor.read_text()

In [4]:
clean_text = preprocessor.clean_text(text=text)

In [5]:
chunks = preprocessor.split_into_chunks(
    text=clean_text, 
    max_chunk_size=600
)

In [6]:
preprocessor.save_chunks(
    chunks=chunks,
    path="/home/murad/Documents/contextual_embeddings/chunks"
)

In [7]:
embedding_model = BGEM3FlagModel(model_name_or_path="BAAI/bge-m3",
                                 use_fp16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [8]:
chunks_embeddings = embedding_model.encode(
    sentences=chunks,
    batch_size=12,
    max_length=8192,
    )['dense_vecs']

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [9]:
collection = preprocessor.create_database(
    chunks=chunks,
    embeddings=chunks_embeddings
)

Using ChromaDB directory: /tmp/tmpanlo7h2f
Successfully stored 37 chunks with embeddings.
Database location: /tmp/tmpanlo7h2f


In [14]:
query = "Turinq testi nədir?"
query_embedding = embedding_model.encode(
    sentences=query,
    batch_size=12,
    max_length=1024
)["dense_vecs"]
query_embedding = query_embedding.tolist()

In [15]:
collection.query(
    query_embeddings=query_embedding,
    n_results=10,
    include=["distances", "metadatas", "documents"]
)

{'ids': [['chunk_16',
   'chunk_17',
   'chunk_15',
   'chunk_26',
   'chunk_29',
   'chunk_35',
   'chunk_18',
   'chunk_22',
   'chunk_21',
   'chunk_1']],
 'embeddings': None,
 'documents': [['Turinq, bir kompüterin süni zəkaya sahib ola biləcəyini və hakimi çaşdıracaq qədər inandırıcı ola biləcəyini iddia edir. Hakim, insanla yoxsa kompüterlə danışdığını anlamayacaq. Bu testə Turinq testi deyilir.Turinq testinin məqsədi və əhəmiyyəti Turinq testi, bir maşının insan səviyyəsində zəkaya sahib olub olmadığını ölçmək üçün bir meyardır. Testin məqsədi, bir maşının davranışının insan davranışına o qədər bənzəyib-bənzəmədiyini müəyyən etməkdir ki, onu ayırd etmək mümkün olmasın.',
   'Turinq testi, süni zəka tədqiqatları sahəsində mühüm bir rol oynamış və bu sahədə bir çox müzakirəyə səbəb olmuşdur. Turinq testinin tənqidləri Turinq testi, süni zəkanın ölçülməsi üçün bir meyar olaraq tənqid edilmişdir. Bəzi tənqidçilər, testin yalnız bir maşının insan kimi davranma qabiliyyətini ölçdüyünü